In [7]:
import requests
import os 
import json
import pandas as pd
!pip install cve_searchsploit
import cve_searchsploit as CS


Initial Extraction

In [8]:
#Atualiza  a base de exploits
CS.update_db()

Refreshing exploit-database repo with latest exploits


From https://gitlab.com/exploit-database/exploitdb
 * branch                main       -> FETCH_HEAD
  0% (0 of 46494) |                      | Elapsed Time: 0:00:00 ETA:  --:--:--


Already up to date.
Refreshing EDBID-CVE mapping


 91% (42375 of 46494) |################  | Elapsed Time: 0:00:00 ETA:   0:00:00
100% (46494 of 46494) |##################| Elapsed Time: 0:00:00 Time:  0:00:00


In [9]:
payload = {}
DTRACK_API_KEY = os.getenv("DTRACK_API_KEY", default=None)
headers = {
  'X-Api-Key': DTRACK_API_KEY
}
base_url = "http://localhost:8081"

#Usado para filtrar as vulnerabilidades analisadas
EPSS_SCORE_THRESHOLD=0.2

In [10]:
#Obtém os projetos

offset=0
totalProjects = 1
while(offset < totalProjects):
    url = base_url+"/api/v1/project?offset="+str(offset)
    response = requests.request("GET", url, headers=headers, data=payload)
    totalProjects = int(response.headers["X-Total-Count"])
     
    projects_response = response.json()
    offset +=100


In [23]:
projects = []
for project in projects_response:
    project = {"name-version":project["name"]+"-"+project["version"],"id":project["uuid"]}
    projects.append(project)

file_path="./sankey/data/projects.json"
os.makedirs(os.path.dirname(file_path), exist_ok=True)

with open(file_path, "w+") as json_file:
    json.dump(projects, json_file)

In [19]:
components = []
for project in projects: 
    offset=0 
    totalcomponets = 1
    while(offset < totalcomponets):
        url = base_url+"/api/v1/component/project/"+project['id']+"?offset="+str(offset)
        response = requests.request("GET", url, headers=headers, data=payload)
        totalcomponets = int(response.headers["X-Total-Count"])
        offset+=100
        components_response = response.json()
        for component in components_response:
            component = {"component_name": component["name"], "component_version": component["version"],"component_id": component["uuid"], "project_id":component["project"]["uuid"]}
            components.append(component)  

file_path="./sankey/data/components.json"
os.makedirs(os.path.dirname(file_path), exist_ok=True)

with open(file_path, "w+") as json_file:
    json.dump(components, json_file)

In [20]:
vulnerabilities = []
for component in components:
    offset=0
    totalvulnerabilities=1
    while(offset < totalvulnerabilities):
        url = base_url+"/api/v1/vulnerability/component/"+component["component_id"]+"?offset="+str(offset)
        response = requests.get(url, headers=headers, data=payload)
        offset+=100
        totalvulnerabilities = int(response.headers["X-Total-Count"])
        vulnerabilities_json = response.json()
    if vulnerabilities_json != []:
        for vulnerability in vulnerabilities_json: 
            if "epssScore" in vulnerability:
                vulnerability = {"component_id": component["component_id"], "vulnerability_id": vulnerability["vulnId"], "epssScore": vulnerability["epssScore"]}
            else:
                vulnerability = {"component_id": component["component_id"], "vulnerability_id": vulnerability["vulnId"], "epssScore": 0.0}
            vulnerabilities.append(vulnerability)

file_path="./sankey/data/vulnerabilities.json"
os.makedirs(os.path.dirname(file_path), exist_ok=True)

with open(file_path, "w+") as json_file:
    json.dump(vulnerabilities, json_file)

In [21]:

vulnerabilities_ids = set()
for vulnerability in vulnerabilities:
    vulnerabilities_ids.add(vulnerability["vulnerability_id"])

In [22]:
exploits = {}
for vulnerability_id in vulnerabilities_ids:
    exploit_response = CS.edbid_from_cve(vulnerability_id)
    if exploit_response is not None: 
        for exploit in exploit_response:
            if vulnerability_id not in exploits:
                exploits[vulnerability_id]=[exploit]
            else:
                exploits[vulnerability_id].append(exploit)
        
file_path="./sankey/data/exploits.json"
os.makedirs(os.path.dirname(file_path), exist_ok=True)

with open(file_path, "w+") as json_file:
    json.dump(exploits, json_file)

In [30]:
systems_names=[]
for project in projects: 
    systems_names.append(project["name-version"])

file_path="./sankey/data/systems_names.csv"
df = pd.DataFrame(systems_names, columns=["Nome"])
df.to_csv(file_path, index=False,header=False)


['juice-shop-1.0.0', 'keycloak-1.0.0']
